In [1]:
# Generic stuff
from IPython.display import IFrame

#Documentation : http://docs.scipy.org/doc/scipy/reference/sparse.html
"""
Sparse matrices can be used in arithmetic operations: they support addition, subtraction, multiplication, division, and matrix power.
Advantages of the CSR format

        efficient arithmetic operations CSR + CSR, CSR * CSR, etc.
        efficient row slicing
        fast matrix vector products
Disadvantages of the CSR format

        slow column slicing operations (consider CSC)
        changes to the sparsity structure are expensive (consider LIL or DOK)
"""
import scipy.sparse as sparse
import scipy.sparse.csgraph as csgraph
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix
from scipy.sparse import dia_matrix
import scipy.sparse.linalg as spl

import scipy as sc
import scipy.misc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Signal processing on graph

### Introduction

First, we would like to thank John D Cook for its [short introduction](https://www.johndcook.com/blog/2016/02/09/fourier-transform-of-a-function-on-a-graph/) on Fourier series on graph, and of course [Pierre Vandergheynst](https://scholar.google.ch/citations?user=1p9NOFEAAAAJ&hl=fr), for its interesting courses and its contributions to this field.

Also, you'll find interesting references in this book : Lectures on Spectral Graph Theory by FanR.K.Chung: http://www.math.ucsd.edu/~fan/cbms.pdf

In [2]:
#IFrame("doc/SpectralGraphTheory/cbms.pdf", width=1200, height=800)

Here are two very good references from Petar Veličković :
* [Theoretical Foundations of Graph Neural Networks](https://www.youtube.com/watch?v=uF53xsT7mjc&ab_channel=PetarVeli%C4%8Dkovi%C4%87)
* [Intro to graph neural networks (ML Tech Talks](https://www.youtube.com/watch?v=8owQBFAHw7E&ab_channel=TensorFlow)
* [Recent set of references](https://twitter.com/PetarV_93/status/1306689702020382720)
* [Some introduction to Graph attention network](https://petar-v.com/GAT/)

### Fourier series

Fourier series properties can be analyzed within various mathenatical framework. One of them is related to the Laplacian operator.

We recall that the Laplacian operator, sometimes written $\nabla \cdot \nabla$, $\nabla^2$ or $\Delta$, where $\nabla$ can be written $\left( \frac{\partial}{\partial x_0}, \frac{\partial}{\partial x_1}, \dots \frac{\partial}{\partial x_{n-1}} \right)$ and the laplacian operator applied to a function $f$ reads $\Delta f = \sum_{i=0}^{n-1} \frac{\partial f}{\partial x_i^2}$.

It is interesting to notice that the trigonometric polynomial that defines the (separable) Fourier series basis elements are eigenfunctions for the laplacian operators over euclidean spaces:

lets define $\delta_{nF}(x) = e^{2 \pi j nFx}$

And
\begin{align}
  \frac{\partial \delta_{nF}}{\partial x} (x) &= \frac{\partial  e^{2 \pi j nFx}}{\partial x} \\
  &= 2 \pi j nF e^{2 \pi j nFx} \\
\end{align}

And
\begin{align}
  \frac{\partial \delta_{nF}}{\partial x^2} (x) &=  -(2\pi nF)^2 e^{2 \pi j nFx} \\
  &= -(2\pi nF)^2 \delta_{nF}\\
  &= \lambda \delta_{nF}
\end{align}

Moreover, eigenfunctions of the laplacian can also be defined for functions on finite domains (Bessel functions for radially symmetric domain like disks and sphere)

## Elements of calculus on graphs

### Definitions, operators
Graphs are not euclidean spaces, lets define some calculus elements before jumping on more advanced stuff:

| Name | ........................ Definition ........................ | Comment |
|:----:|:----------:|:-------:|
| Graph | $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ | |
| Vertices | $\mathcal{V} = {1,\dots, n}$ | n is the cardinality of the vertices |
| Vertex weights | $b_i > 0 \forall i \in \mathcal{V}$| |
| Edges | $\mathcal{E} \subseteq \mathcal{V} \times \mathcal{V}$ | |
| Edge weights | $a_{ij} \geq 0 \quad \forall \quad (i,j) \quad \in \mathcal{E}$ | |
| Vertex field | $L^2(\mathcal{V}) = {f : \mathcal{V} \rightarrow \mathbb{R}^n}$ | Functional space of all $L^2$-integrable functions <br> sampled over the graph (1 real value per vertex) |
| graph function | $f = (f_1, \dots, f_n)$ | One element of $L^2(\mathcal{V})$ presented above  |
| Inner product | $\langle f,g \rangle_{L^2(\mathcal{V})} = \sum_{i \in \mathcal{V}} b_i f_i g_i$ | Allows to define a functional Hilbert <br> space of graph-based functions |
| Gradient operator | $(\nabla f)_{ij} = \sqrt{a_{ij}}(f_i-f_j)$ | $\nabla : L^2(\mathcal{V}) \rightarrow L^2(\mathcal{E})$ |
| Divergence operator | $(div F)_{i} = \frac{1}{b_i} \sum_{j:(i,j) \in \mathcal{E}} \sqrt{a_{ij}}(F_{ji}-F_{ij})$ |$div : L^2(\mathcal{E}) \rightarrow L^2(\mathcal{V})$ |
| Gradient Adjoint |$\nabla^{\star}F = -div F$ | $\langle F,\nabla f \rangle_{L^2(\mathcal{E})} = \langle \nabla^{\star}F, f \rangle_{L^2(\mathcal{V})} = \langle -div F, f \rangle_{L^2(\mathcal{V})}$ |
| Weight matrix | $A = (a_{ij})$ | | |
| Degree matrix | $D = diag\left(d_i\right)$ where $d_i=\sum_{j\neq i}a_{ij})$ | | |
| (Unnormalized) Laplacian operator | $(\Delta F)_{i} = \frac{1}{b_i} \sum_{j:(i,j) \in \mathcal{E}} a_{ij}(f_i-f_j)$ | $\Delta : L^2(\mathcal{V}) \rightarrow L^2(\mathcal{V})$, this one is $\Delta = D - A$ |
| Normalized Laplacian | $(\Delta F)_{i} = \sum_{j:(i,j) \in \mathcal{E}} \frac{1}{b_i\sqrt{d_i d_j}} a_{ij}(f_i-f_j)$ | $\Delta : L^2(\mathcal{V}) \rightarrow L^2(\mathcal{V})$, this one is $\Delta = I - D^{-\frac{1}{2}} A D^{-\frac{1}{2}}$ |
| Random walk Laplacian | $\Delta = I - D^{-1} A $ |  $\Delta : L^2(\mathcal{V}) \rightarrow L^2(\mathcal{V})$, this one is $\Delta = I - D^{-1} A $ |

Those definitions are coming from slides from Xavier Bresson.

### Some precision about finite difference operators

#### Gradient
It must be noticed that gradient operator is a finitie difference operator of type forward difference in case we consider directed graphs.
It is positively valued if the graph function has higher value at the terminal vertex than at the initial vertex.

To get an idea of what the gradient linear operator looks like, we tried to draw it:

\begin{align*}
    \begin{pmatrix}
        0 & 0 & 0 & \cdots & 0 \\
        \sqrt{a_{0,1}} & -\sqrt{a_{0,1}} & 0 & & 0 \\
        \sqrt{a_{0,2}} & 0 & -\sqrt{a_{0,2}} & & 0 \\
        \vdots & & & & \\
        \sqrt{a_{0,I-1}} & 0 & 0 & \cdots & -\sqrt{a_{0,I-1}}\\
        -\sqrt{a_{1,0}} & \sqrt{a_{1,0}} & 0 & \cdots & 0 \\
        0 & 0 & 0 & & 0 \\
        0 & \sqrt{a_{1,2}} & -\sqrt{a_{1,2}} & & 0 \\
        \vdots & & & & \\
        0 & \sqrt{a_{1,I-1}} & 0 & & -\sqrt{a_{1,I-1}} \\
        -\sqrt{a_{2,0}} & 0 & \sqrt{a_{0,2}} & & 0 \\
        0 & -\sqrt{a_{2,1}} & \sqrt{a_{2,1}} & & 0 \\
        0 & 0 & 0 & & 0 \\
        \vdots & & & & \\
        0 & 0 & \cdots & -\sqrt{a_{I-1,I-2}} & \sqrt{a_{I-1,I-2}}\\
        0 & 0 & 0 & & 0
    \end{pmatrix}
\end{align*}

#### Divergence
We must notice, that we prefered the following notation for the divergence operator (although it is exactly the same thing):

| Name | ............................. Definition ............................. | ...............Comment............... |
|:----:|:----------:|:-------:|
| Divergence operator | $(div F)_{i} = \sum_{(k,l) \in \mathcal{E}} \sqrt{a_{kl}}(\delta_{i}(l)-\delta_{i}(k)) F_{kl}$ |$div : L^2(\mathcal{E}) \rightarrow L^2(\mathcal{V})$ |

Its matrix would look like that:

\begin{align*}
    \begin{pmatrix}
        0 & -\sqrt{a_{0,1}} & -\sqrt{a_{0,2}} & \cdots & -\sqrt{a_{0,I-1}} & \sqrt{a_{1,0}} & 0 &0 & \cdots & 0 & \sqrt{a_{2,0}} & 0 & 0 & \cdots & 0 & 0\\
        0 & \sqrt{a_{0,1}} & 0 &  \cdots & 0 & -\sqrt{a_{1,0}} & 0 & -\sqrt{a_{1,2}} & \cdots & -\sqrt{a_{1,I-1}} & 0 & \sqrt{a_{2,1}} &  0 & & 0 & 0\\
        0 & 0 & \sqrt{a_{0,2}} & \cdots & 0 & 0 & 0 & \sqrt{a_{1,2}} & \cdots & 0 & -\sqrt{a_{0,2}} &  -\sqrt{a_{2,1}} &  0 & & \vdots &  0\\
        \vdots &  & \cdots &  & \vdots &  &  & \cdots & & & & & & & \sqrt{a_{I-1,I-2}} &  0\\
        0 & 0 & 0 & \cdots & \sqrt{a_{0,I-1}} & 0 & 0 & 0 & \cdots & \sqrt{a_{1,I-1}} & 0 & 0 &  0 & \cdots & -\sqrt{a_{I-1,I-2}} & 0\\
    \end{pmatrix}
\end{align*}

#### Laplacian
We recall that laplacian operator $\Delta$ is usually expressed as
\begin{align*}
    \Delta f &= \nabla^2 f \\
    &= \nabla \cdot \nabla f \\
    &= \nabla^{T} \nabla f \\
    &= -div(grad(f))
\end{align*}

In [119]:
#lets define some utilities for testing on random/symmetric sparse matrices

# define graph adjacency matrix as a random csr matrix
def gen_random_spm(size):
    return sparse.rand(size[0], size[1], density=0.9, format='csr', dtype=np.float32, random_state=None)

def gen_random_graph(size):
    A = gen_random_spm((size,size))
    # Now make it symmetric
    return A+A.T-sparse.diags(A.diagonal())

# define graph adjacency matrix as a small symmetric csr matrix
def gen_small_graph():
    weight=[4]
    rowIdx=[0]
    colIdx=[1]
    A = csr_matrix((np.array(weight), (np.array(rowIdx), np.array(colIdx))),dtype=np.float32, shape=(2, 2))
    #Build a symmetric matrix
    return A+A.T-sparse.diags(A.diagonal())

def gen_iota_graph():
    weight=[1,2,3,4]
    rowIdx=[0,0,1,1]
    colIdx=[0,1,0,1]
    return csr_matrix((np.array(weight), (np.array(rowIdx), np.array(colIdx))),dtype=np.float32, shape=(2, 2))

In [102]:
# lets define some operators, elementwise
def grad(G,f):
    """-G is the graph adjacency matrix
       -f is the function defined on this graph
       Output has the dimension of the adjacency matrix
       This operator can be considered a forward difference operation
       Elementwise version
    """
    g = lil_matrix(G.shape)
    #iterate over all edges, row_index is i, col_idx is j
    for row_idx, col_idx, weight in zip(*G.nonzero(), G.data):
        g[row_idx, col_idx] = np.sqrt(weight)*(f[row_idx]-f[col_idx])
    return g

def div(G,F,b=None):
    """-G is the graph adjacency matrix
       -F is the gradient of function f on this graph
       -b is the optional vertex_weight
       Output is 1D and has its size equal to the cardinality of the vertices
       Elementwise version
    """
    # If b is not provided, assume all vertices have weight 1
    if b is None:
        v_weight = lambda idx: 1
    else:
        v_weight = lambda idx: 1/b[idx]
        
    def delta_i(i):
        return lambda x: 1 if x==i else 0
    
    div = np.zeros(G.shape[0])
    for row_idx in range(div.size):
        d = 0
        for g_row_idx, g_col_idx, weight in zip(*G.nonzero(), G.data):
            #print('row_idx is {}, col_idx is {}, weight is {}'.format(g_row_idx, g_col_idx, weight))
            #print('F is {}, delta+ is {}, delta- is {}'.format(F[g_row_idx, g_col_idx],
            #                                                   delta_i(row_idx)(g_row_idx),
            #                                                   delta_i(row_idx)(g_col_idx)))
            d += np.sqrt(weight)*F[g_row_idx, g_col_idx]*(delta_i(row_idx)(g_col_idx)-delta_i(row_idx)(g_row_idx))
            #d += np.sqrt(weight)*(F[row_idx, col_idx]-F[col_idx,row_idx])
        div[row_idx] = d#v_weight(row_idx)*d
    return div

def laplacian(G,f,b=None):
    """-G is the graph adjacency matrix
       -f is a funcion defined on this graph
       -b is the optional vertex_weight
       Output is 1D and has its size equal to the cardinality of the vertices
       Elementwise version
    """
    lap = np.zeros(G.shape[0])
    for row_idx in range(lap.size):
        row = G.getrow(row_idx)
        l = 0
        for _, col_idx, weight in zip(*row.nonzero(), row.data):
            l += weight*(f[row_idx]-f[col_idx])
        lap[row_idx] = l#v_weight(row_idx)*d
    return lap

Now, we are going to define the operators as linear operators. To do so, we will need to defined the incidence matrix (actually a weighted version of it).

We recall the definition of the incidence matrix $\nabla$:

\begin{align*}
    \nabla_{e,v} &= \begin{cases}
        1 \; \text{if} \; v \; \text{is initial vertex of edge} \; e\\
        -1 \; \text{if} \; v \; \text{is terminal vertex of edge} \; e\\
        0 \; \text{if} \; e \; \text{is not incident on vertex } \; v\\
    \end{cases}
\end{align*}

This operator can be considered as a forward difference operator.

In [103]:
# Now define those operators as linear operators

def get_incident_matrix(G):
    """-G is the graph adjacency matrix
    Returns the matrix that takes a function f, valued of the the network and returns the gradient
    (as a linear index (0,0) (0,1) (0,2)... (1,0) (1,1) (1.2)...
    """
    grad_mat = lil_matrix((np.prod(G.shape), G.shape[0]), dtype=np.float32)
    #print('G shape is {} gradmat shape is {}'.format(G.shape,grad_mat.shape))
    for i, j, weight in zip(*G.nonzero(), G.data):
        weight = np.sqrt(weight)
        grad_mat[i*G.shape[1]+j,i]+=weight #+1 if non weighted
        grad_mat[i*G.shape[1]+j,j]-=weight #-1 if non weighted
    return grad_mat

def get_degree_matrix(G):
    """-G is the graph adjacency matrix
    Returns the degree matrix of the graph
    """
    diag = np.squeeze(np.array(G.sum(axis=1)))
    return sparse.diags(diag)

def grad_with_matrix(G,f):
    """-G is the graph adjacency matrix
       -f is the function defined on this graph
       Output has the dimension of the adjacency matrix
       This operator can be considered a forward difference operation
       matrix version
    """
    return get_incident_matrix(G).dot(lil_matrix(f.reshape(f.size,1))).tolil().reshape(G.shape)

def div_with_matrix(G,F,b=None):
    """-G is the graph adjacency matrix
       -F is the gradient of function f on this graph
       -b is the optional vertex_weight
       Output is 1D and has its size equal to the cardinality of the vertices
       matrix version
    """
    d = (-get_incident_matrix(G).T).dot(F.tolil().reshape((np.prod(G.shape),1)))
    return np.squeeze(np.array(d.todense()))

def laplacian_with_matrix(G,f,b=None):
    """-G is the graph adjacency matrix
       -f is a funcion defined on this graph
       -b is the optional vertex_weight
       Output is 1D and has its size equal to the cardinality of the vertices
       matrix version
    """
    return (get_degree_matrix(G)-G).dot(f)

In [104]:
# check if both grad give the same result:
size = 20
G1 = gen_random_graph(size)
#G1 = gen_small_graph()
f1 = np.random.rand(size)
grad1 = grad(G1, f1)
gradm1 = grad_with_matrix(G1, f1)
assert(np.allclose(grad1.todense(), gradm1.todense()))

In [105]:
# check if both div give the same result:
size = 20
G1 = gen_random_graph(size)
F1 = gen_random_graph(size)
div1 = div(G1, F1)
divm1 = div_with_matrix(G1, F1)
assert(np.allclose(div1, divm1, atol=1e-4))

In [108]:
# check if both laplacian give the same result
size = 20
G1 = gen_random_graph(size)
f1 = np.random.rand(size)
lap1 = laplacian(G1, f1)
lapm1 = laplacian_with_matrix(G1, f1)
assert(np.allclose(lap1, lapm1))

Check the following assumption, for a given linear operator A, and its transpose B, where we cannot explicitly obtain $A^t$ neither $B^t$:

\begin{align*}
    xA^t A y &= x^tBAy \\
    x^t \cdot (A^tA y) &= x^t \cdot (BAy) \\
    (x^tA^t) \cdot (A y) &= (x^tB) \cdot (Ay) \\
    (Ax)^t \cdot (Ay) &= (B^tx) \cdot (Ay)
\end{align*}

By comparing $a = x^t \cdot (BAy)$ and $b = (Ax)^t \cdot (Ay)$ that both give a scalar

In [116]:
# check if matrix grad and matrix div are transpose of each other (by definition they should...)
f1 = np.random.rand(size)
f2 = np.random.rand(size)
a = f1.dot(-div_with_matrix(G1, grad_with_matrix(G1, f2)))
b = grad_with_matrix(G1, f1).multiply(grad_with_matrix(G1, f2)).sum()
assert(np.allclose(a,b))

In [117]:
# check if grad and div are transpose of each other
f1 = np.random.rand(size)
f2 = np.random.rand(size)

a = f1.dot(-div(G1, grad(G1, f2)))
b = grad(G1, f1).multiply(grad(G1, f2)).sum()
assert(np.allclose(a,b))

In [118]:
# Check if div(grad()) for matrix based implementation gives laplacian
G1 = gen_random_graph(size)
#G1 = gen_small_graph()
f1 = np.random.rand(size)
#f1 = np.random.rand(2)
a = -div(G1, grad(G1, f1))
print(a)
b = laplacian(G1, f1)
print(b)
c = csgraph.laplacian(G1)
print(c.dot(f1))
assert(np.allclose(a,b))

[-14.18685254 -11.46867769 -17.00218327  10.39833276 -18.7286261
  -1.93604717  -8.69733653  13.4095348   -4.05484683  14.60066756
   9.94081769  -3.71789777  12.25901727   9.55552775  14.60443636
   7.4685563  -11.21909322  -2.27185562  -6.73175711   7.77828333]
[-7.09342632 -5.73433903 -8.5010918   5.19916655 -9.36431314 -0.96802357
 -4.34866828  6.70476759 -2.02742345  7.30033387  4.970409   -1.85894895
  6.1295084   4.77776394  7.30221818  3.7342782  -5.60954655 -1.13592778
 -3.36587855  3.88914171]
[-7.09342603 -5.73433827 -8.50109178  5.19916642 -9.36431299 -0.96802239
 -4.34866833  6.70476968 -2.02742277  7.30033465  4.97040805 -1.85894912
  6.12950938  4.77776382  7.30221649  3.73427867 -5.6095466  -1.13592795
 -3.36587824  3.88914024]


AssertionError: 

In [66]:
# Check if div(grad()) for matrix based implementation gives laplacian
f1 = np.random.rand(size)

a = div_with_matrix(G1, grad_with_matrix(G1, f1))
b = laplacian_with_matrix(G1, f1)
assert(np.allclose(a,b))

ValueError: dimension mismatch

### Studying graph thanks to those operators

It is interesting to notice that the concept of smoothness/regularity can be extended to graph structured data as well as:

smoothness($f$) =
\begin{align}
  & \quad \langle f, \Delta f \rangle_{L^2(\mathcal{V})} \\
  = &\sum_{(i,j) \in \mathcal{E} \; \text{s.t} \; i\neq j} (f_i-f_j)^2 \\
  \geq & \qquad 0
\end{align}

### Fourier series on graph

Following the definition seen in the previous section, we can see that on cane define eigenfunctions of laplacian operator on various domain including graph, for instance in our case, where we consider our laplacian operator $L$ as: $L=D^{-\frac{1}{2}}AD^{-\frac{1}{2}}$ where $A$ is the adjacency matrix and $D$ is the degree matrix of the graph.

Hopefully, in an undirected graph, $L$ is a symmetric positive semi definite matrix, hence is diagonalizable in an orthonornmal basis. A function that is defined by a set of values, one per node can then be analyzed over this graph, using eigenvectors of the graph ordered by the norm of their corresponding eigenvalues.

On can then consider that projection of the function over the lower eigenvectors corresponds to low "frequencies", and conversely, the high "frequencies" are obtained when projecting the function over the highest eigenvalues


In [ ]:
# Import an image, and downsize it in order to make the problem numerically tractable
im = sc.misc.imresize(sc.misc.ascent(),size=(128,128))
im = im.astype(np.float32)
plt.imshow(im, interpolation='nearest', cmap=cm.gray, vmin=0, vmax=255)
figsize(8,8)
plt.axis("off")
colorbar()       # displays the color bar close to the image
title('Initial image')

In [ ]:
#Threshold: valued used to forbid disconnected nodes in the graph of the image
thresh= 1e-4

#Sigma of the gaussian distance metric
sigma = 2
sigmaSquare = sigma**2

#Size of the stencil
stenSize = 4

#will contain the value of the edges
data=[]
rowIdx=[]
colIdx=[]

"""
Now we are going to fill the graph matrix, neglecting small terms.
Distance are actually computed twice, this can be optimized
"""

for idxI in range(im.shape[0]):
  for idxJ in range(im.shape[1]):
    curPixIdx=idxI+im.shape[0]*idxJ
    for stenX in range(-stenSize,stenSize+1):
      for stenY in range(-stenSize,stenSize+1):
        stenIdxX = idxI+stenX
        stenIdxY = idxJ+stenY
        destPixIdx = stenIdxX+im.shape[0]*stenIdxY
        if (stenIdxX>=0)and(stenIdxX<im.shape[0])and\
          (stenIdxY>=0)and(stenIdxY<im.shape[1])and\
          ((stenIdxX!=idxI)and(stenIdxY!=idxJ)):
          dist = np.exp(-np.linalg.norm(im[idxI,idxJ]-im[stenIdxX,stenIdxY])**2/(2*sigmaSquare))
          dist=max(thresh,dist)
          data.append(dist)
          rowIdx.append(curPixIdx)
          colIdx.append(destPixIdx)

"""
   Practical implementation using scipy support for sparse matrices, The compressed sparse row (CSR) format
   represents a matrix M by three (one-dimensional) arrays, that respectively contain:
      -nonzero values
      -the extents of rows
      -column indices
    This format allows fast row access and matrix-vector multiplications (Mx)
"""
A = csr_matrix((np.array(data), (np.array(rowIdx), np.array(colIdx))), dtype=np.float32)

#Now the diagonal matrix
D = dia_matrix((np.array(A.sum(axis=1)),0),shape=A.shape)

#We can define the laplacian: L = D^-0.5 A D^-0.5
L = D.power(-0.5)*A*D.power(-0.5)

In [ ]:
"""
Compute the eigendecomposition of the laplacian of the graph
"""

# Get the first 100 eigenvalues of L, a symmetric positive definite matrix
# See http://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigsh.html and
# http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.linalg.svds.html
[w,X] = spl.eigsh(L, k=100) #size is now [imsize,nbcomponent]

In [ ]:
#Show the spectrum of the image, i.e its expression in the basis made of the first lapacian eigenvectors
spectrum = np.abs(np.dot(X.T,im.reshape(im.size)))
plt.plot(spectrum)

In [ ]:

#Norm of each eigenvector
#print("Norm is "+str(np.linalg.norm(X[:,0])))

for i in np.arange(X.shape[1])[::-1]:
  figure(i)
  imshow(np.reshape(X[:,i],im.shape), interpolation='nearest', cmap=cm.gray)